In [20]:
# https://pro.arcgis.com/en/pro-app/latest/arcpy/get-started/writing-geometries.htm
# https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/get-raster-properties.htm
# https://gis.stackexchange.com/questions/180757/creating-polygon-feature-class-from-x-y-coordinates-using-arcpy
from arcpy.sa import *
arcpy.env.overwriteOutput = True

# Read input raster
raster = "pensacola_after_dem.tif"

# Get raster extent
myRaster = Raster(raster)   # need to read raster as Raster object
myExtent = myRaster.extent

# Store extent coordinates 
xmax = myExtent.XMax
xmin = myExtent.XMin
ymax = myExtent.YMax
ymin = myExtent.YMin

# Store extent values as list of coordinates 
coordinates = [(xmin, ymin),
               (xmin, ymax),
               (xmax, ymax),
               (xmax, ymin)]

# Get coordinate system
sr = arcpy.Describe(raster).spatialReference

# Create a feature class with a spatial reference of GCS WGS 1984
result = arcpy.management.CreateFeatureclass(
    arcpy.env.scratchGDB, 
    "rasterExtent", 
    "POLYGON", 
    spatial_reference=sr
)


# Create feature class
outPolyExtent= result[0]

# Use Insert cursor to add new geometry to feature class Write feature to new feature class
with arcpy.da.InsertCursor(outPolyExtent, ['SHAPE@']) as cursor:
    cursor.insertRow([coordinates])

In [7]:
selected_features = arcpy.Describe("MHWLTransects").FIDSet
print(selected_features)
query_list = f"({','.join([val for val in selected_features.split(';')])})"
query_list

0


'(0)'

In [20]:
aprx = arcpy.mp.ArcGISProject("CURRENT")
m = aprx.listMaps()
for i in m:
    print(i.name)
    if i.name=="PensacolaBeach":
        i.addDataFromPath(r"G:\My Drive\Projects\FlSeaGrantHurricaneViewsheds\Data\PensacolaBeach\ObserverPoints.shp")

Map
PensacolaBeach
VillanoBeach


In [17]:
# https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/generate-points-along-lines.htm
arcpy.management.GeneratePointsAlongLines(
    Input_Features= lineInput,
    Output_Feature_Class= pointOutput,
    Point_Placement="PERCENTAGE",
    Percentage=pointSpacing,
    Include_End_Points="NO_END_POINTS",
)

<Result 'G:\\My Drive\\Projects\\FlSeaGrantHurricaneViewsheds\\Data\\PensacolaBeach\\ObserverPoints.shp'>

In [1]:
#parameters to modify:
# lineInput = "line_Select" jdm: I believe this can just be the selected line
# in our case the CCCL or MHWL line. Although if we have an area with multiple lines we can modify 
# script do the detect selection fucntionality
workspace = r"G:\My Drive\Projects\FlSeaGrantHurricaneViewsheds\Data\MexicoBeach"
lineInput =  "MB_MHWL"
# pointOutput = workspace + "\ObserverPoints"
# pointSpacing = 10
pointInput = "MB_Transect0"
rasterInput = "MexicoCity_after_dem.tif"
observerElevation = "6"
viewDistance = "2 Miles"
outRaster = workspace + "\Viewsheds\Viewshed_MB0" #FL_G3??

# Generate points along line, 
# Only the part of the viewshed line to that is accounted for w/in the extent of the input raster
# arcpy.env.workspace = arcpy.env.scratchGDB
# clip_line = arcpy.analysis.Clip( "MHWL",
#                                 "rasterExtent",
#                                "clip_line")

In [3]:
#geodesic viewshed
#Get projection from MHWL data and put in variable e.g. outCoordSys

#NAD903 = 'PROJCS["NAD_1983_2011_StatePlane_Florida_North_FIPS_0903_Ft_US",GEOGCS["GCS_NAD_1983_2011",DATUM["D_NAD_1983_2011",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic"],PARAMETER["False_Easting",1968500.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-84.5],PARAMETER["Standard_Parallel_1",29.58333333333333],PARAMETER["Standard_Parallel_2",30.75],PARAMETER["Latitude_Of_Origin",29.0],UNIT["Foot_US",0.3048006096012192]],VERTCS["NAD_1983_2011",DATUM["D_NAD_1983_2011",SPHEROID["GRS_1980",6378137.0,298.257222101]],PARAMETER["Vertical_Shift",0.0],PARAMETER["Direction",1.0],UNIT["Meter",1.0]]'
# We can also just get the projection from one of our input files that we are already sure is in the right projection
feature_class = "MB_MHWL"
desc = arcpy.Describe(feature_class)
sr = desc.spatialReference
print(sr.name)  # Prints the name of the projection
print(sr.factoryCode)  # Prints the factory code of the projection

# Since the project is in feet let's keep the parameters in feet as well e.g. 6 feet on observer elevation
with arcpy.da.SearchCursor(pointInput,['SHAPE@','FID']) as cursor:
    for row in cursor:
        print("Running viewshed number {}".format(row[1]))
        with arcpy.EnvManager(outputCoordinateSystem=sr, parallelProcessingFactor="100%", scratchWorkspace=workspace):
            out_raster = arcpy.sa.Viewshed2(
                in_raster=rasterInput,
                in_observer_features=row[0],
                out_agl_raster=None,
                analysis_type="FREQUENCY",
                vertical_error="0 Meters",
                out_observer_region_relationship_table=None,
                refractivity_coefficient=0.13,
                surface_offset="0 Meters",
                observer_elevation=observerElevation,
                observer_offset="1 Meters",
                inner_radius=None,
                inner_radius_is_3d="GROUND",
                outer_radius="1 Miles",
                outer_radius_is_3d="GROUND",
                horizontal_start_angle=0,
                horizontal_end_angle=360,
                vertical_upper_angle=90,
                vertical_lower_angle=-90,
                analysis_method="ALL_SIGHTLINES",
                analysis_target_device="CPU_ONLY"
            )
            out_raster.save(outRaster + str(row[1]) +".tif")

NAD_1983_StatePlane_Florida_North_FIPS_0903_Feet
2238
Running viewshed number 0
Running viewshed number 1
Running viewshed number 2
Running viewshed number 3
Running viewshed number 4
